In [ ]:
# Original posts: https://forum.image.sc/t/2d-3d-integer-labels/38732/7
# Code written by @kapoorlab and @VolkerH 

In [13]:
from tifffile import imread
import matplotlib.pyplot as plt
import numpy as np

from skimage import measure
from scipy import spatial
import tqdm
from skimage.segmentation import relabel_sequential
%gui qt



In [14]:
label_slices = imread("C:/Users/Alexis/Desktop/Stack.tif")


In [15]:
def RelabelZ_with_fixes(previousImage, currentImage,threshold):
    # This line ensures non-intersecting label sets
    currentImage = relabel_sequential(currentImage,offset=previousImage.max()+1)[0]
    # I also don't like modifying the input image, so we take a copy
    relabelimage = currentImage.copy()
    waterproperties = measure.regionprops(previousImage, previousImage)
    indices = [prop.centroid for prop in waterproperties] 
    labels = [prop.label for prop in waterproperties]
    if len(indices) > 2:
       tree = spatial.cKDTree(indices)
       currentwaterproperties = measure.regionprops(currentImage, currentImage)
       currentindices = [prop.centroid for prop in currentwaterproperties] 
       currentlabels = [prop.label for prop in currentwaterproperties] 
       if len(currentindices) > 2: #why only > : ?
           for i in range(0,len(currentindices)):
               index = currentindices[i]
               print(f"index {index}")
               currentlabel = currentlabels[i] 
               print(f"currentlabel {currentlabel}")
               if currentlabel > 0:
                      previouspoint = tree.query(index)
                      print(f"prviouspoint {previouspoint}")
                      previouslabel = previousImage[int(indices[previouspoint[1]][0]), int(indices[previouspoint[1]][1])]
                      print(f"previouslabels {previouslabel}")
                      if previouspoint[0] > threshold:
                             relabelimage[np.where(currentImage == currentlabel)] = currentlabel
                      else:
                             relabelimage[np.where(currentImage == currentlabel)] = previouslabel
    return relabelimage 

In [16]:


def merge_labels_across_volume(labelvol, relabelfunc, threshold=3):
    nz, ny, nx = labelvol.shape
    res = np.zeros_like(labelvol)
    res[0,...] = labelvol[0,...]
    backup = labelvol.copy() # kapoors code modifies the input array
    for i in tqdm.tqdm(range(nz-1)):
        res[i+1] = relabelfunc(res[i,...], labelvol[i+1,...],threshold=threshold)
        labelvol = backup.copy() # restore the input array
    return res



In [17]:
relabelled_fixes = merge_labels_across_volume(label_slices, RelabelZ_with_fixes)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

index (165.95244215938303, 618.7647814910026)
currentlabel 1
prviouspoint (2.084749136689926, 0)
previouslabels 1
index (195.92433697347894, 721.9461778471139)
currentlabel 2
prviouspoint (0.9751440525743714, 1)
previouslabels 2
index (203.242006615215, 539.3919514884234)
currentlabel 3
prviouspoint (3.2160890654296934, 3)
previouslabels 4
index (296.5896551724138, 613.7262068965517)
currentlabel 4
prviouspoint (1.6325744490201781, 4)
previouslabels 5
index (308.65667311411994, 713.1363636363636)
currentlabel 5
prviouspoint (1.1144273490223047, 5)
previouslabels 6
index (316.35630498533726, 421.70527859237535)
currentlabel 6
prviouspoint (1.5905344582924372, 6)
previouslabels 7
index (364.38761904761907, 514.4561904761905)
currentlabel 7
prviouspoint (2.1596909671617195, 7)
previouslabels 8
index (385.27171215880895, 672.0012406947891)
currentlabel 8
prviouspoint (6.889611624133844, 8)
previouslabels 9
index (393.0976916735367, 293.3252267106348)
currentlabel 9
prviouspoint (1.55402541

 20%|████████████████▊                                                                   | 1/5 [00:00<00:01,  2.96it/s]

index (165.9536082474227, 620.1494845360825)
currentlabel 1
prviouspoint (1.3847035360735933, 0)
previouslabels 1
index (196.77068273092368, 722.3867469879518)
currentlabel 2
prviouspoint (0.9541500453300882, 1)
previouslabels 2
index (203.78692879914985, 538.8730074388948)
currentlabel 3
prviouspoint (0.7524912711025235, 2)
previouslabels 3
index (293.9872521246459, 614.0042492917847)
currentlabel 4
prviouspoint (2.6172140142866662, 3)
previouslabels 5
index (307.90155945419104, 714.4590643274854)
currentlabel 5
prviouspoint (1.5230672203501883, 4)
previouslabels 6
index (315.4493670886076, 420.0506329113924)
currentlabel 6
prviouspoint (1.8868992230959003, 5)
previouslabels 7
index (364.67625231910944, 514.58719851577)
currentlabel 7
prviouspoint (43.91151878183417, 6)
previouslabels 8
index (384.49, 674.6)
currentlabel 8
prviouspoint (29.652949915303576, 8)
previouslabels 8
index (391.7668863261944, 292.9728171334432)
currentlabel 9
prviouspoint (1.376675481953237, 7)
previouslabels

 40%|█████████████████████████████████▌                                                  | 2/5 [00:00<00:00,  3.06it/s]

...

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:00<00:00,  3.10it/s]

index (164.11077844311376, 619.0449101796407)
currentlabel 1
prviouspoint (0.668533315965586, 0)
previouslabels 1
index (196.030749789385, 718.5985678180286)
currentlabel 2
prviouspoint (2.31753992062584, 1)
previouslabels 2
index (200.6692627206646, 537.9776739356179)
currentlabel 3
prviouspoint (2.697130563626241, 2)
previouslabels 3
index (288.4906542056075, 619.9049844236761)
currentlabel 4
prviouspoint (6.953818131872283, 3)
previouslabels 5
index (304.4765625, 715.296875)
currentlabel 5
prviouspoint (88.97998229757864, 6)
previouslabels 8
index (313.3960244648318, 418.2584097859327)
currentlabel 6
prviouspoint (109.58346241834985, 5)
previouslabels 7
index (364.2878787878788, 519.8993783993784)
currentlabel 7
prviouspoint (4.108690935586454, 5)
previouslabels 7
index (384.4948717948718, 677.9410256410256)
currentlabel 8
prviouspoint (2.2349436102596267, 6)
previouslabels 8
index (390.3821548821549, 293.22053872053874)
currentlabel 9
prviouspoint (2.4942653471134526, 7)
previousla

 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:01<00:00,  3.10it/s]

index (168.44166666666666, 614.1333333333333)
currentlabel 1
prviouspoint (6.548295955597325, 0)
previouslabels 1
index (195.50444115470023, 722.0329385640266)
currentlabel 2
prviouspoint (3.474464447927245, 1)
previouslabels 2
index (194.9619140625, 348.4404296875)
currentlabel 3
prviouspoint (137.48159459293763, 5)
previouslabels 6
index (206.38361045130642, 537.1627078384798)
currentlabel 4
prviouspoint (5.77216941246321, 2)
previouslabels 3
index (299.40611664295875, 614.3022759601707)
currentlabel 5
prviouspoint (12.269379053084176, 3)
previouslabels 4
index (307.9734693877551, 713.034693877551)
currentlabel 6
prviouspoint (4.164831474668974, 4)
previouslabels 5
index (320.5670391061453, 425.93575418994413)
currentlabel 7
prviouspoint (10.505478003581652, 5)
previouslabels 6
index (363.4845261121857, 513.472920696325)
currentlabel 8
prviouspoint (6.476475440444277, 6)
previouslabels 7
index (382.9778761061947, 665.79203539823)
currentlabel 9
prviouspoint (12.243334506538911, 7)
pr

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.13it/s]
